In [40]:
from poke_env.environment.pokemon import Pokemon
import json
from typing import Optional

In [18]:
with open("../../pkmn_data/crit_chance.json", "r") as f:
    crit_chance = json.load(f)

In [19]:
for pkmn in crit_chance.keys():
    if " " in pkmn:
        print(pkmn)

In [41]:
p = Pokemon(1, species="mrmime")
crit_chance["mrmime"]
p.base_stats

{'atk': 45, 'def': 65, 'hp': 40, 'spa': 100, 'spd': 100, 'spe': 90}

In [42]:
def embed_pokemon(pkmn: Optional[Pokemon] = None, revealed = True, fainted = False):
    if not revealed:
        return [0] * 9
    
    if fainted:
        return [0] * 8 + [1]
    
    stats = [pkmn.current_hp_fraction, 
             pkmn.stats["spe"] / 200 if pkmn.stats["spe"] else 0]
    
    status_order = ['par', 'slp', 'frz', 'brn', 'psn']
    status = [0] * len(status_order)
    if pkmn.status:
        status[status_order.index(str(pkmn.status.name).lower())] = 1
        
    crit_chance = (pkmn.base_stats["spe"] / 512)
    
    return stats + status + [crit_chance, int(pkmn.revealed)]

In [34]:
len(embed_pokemon(p, crit_chance))


9

In [43]:
def embed_side(pkmn):
    boosts_order = ['atk', 'spa', 'def', 'spd', 'spe', 'accuracy']
    boosts = [0] * len(boosts_order)
    if pkmn.boosts:
        for stat, boost in pkmn.boosts.items():
            if stat == "evasion":  # evasion clause
                continue
            boosts[boosts_order.index(stat)] = boost
            
    volatile_statuses = ['confusion', 'substitute', 'reflect']
    volatile_status = [0] * len(volatile_statuses)
    for effect in pkmn.effects:
        if effect.name.lower() in volatile_statuses:
            volatile_status[volatile_statuses.index(effect.name.lower())] = 1
            
    battle_info = [int(pkmn.must_recharge), int(pkmn.preparing)]
    
    return boosts + volatile_status + battle_info

In [44]:
len(embed_side(p))

11